In [ ]:
import sys, time, os, asyncio, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.time import Time, TimeDelta
from lsst_efd_client import EfdClient
from lsst.summit.utils import dayObsIntToString
from lsst.summit.utils.efdUtils import calcNextDay


In [ ]:
client = EfdClient('usdf_efd')

# First get the data
## Look at successive weekends with no venting

In [ ]:
startDay1 = 20240316
endDay1 = 20240318
startString1 = dayObsIntToString(startDay1)
endString1 = dayObsIntToString(endDay1)
start1 = Time(f"{startString1}T00:00:00", scale='utc')
end1 = Time(f"{endString1}T04:00:00", scale='utc')
startDay2 = 20240323
endDay2 = 20240325
startString2 = dayObsIntToString(startDay2)
endString2 = dayObsIntToString(endDay2)
start2 = Time(f"{startString2}T00:00:00", scale='utc')
end2 = Time(f"{endString2}T04:00:00", scale='utc')


In [ ]:
m2Temp1 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem3', 'sensorName'],  start1, end1, index=201)
m2Temp1 = m2Temp1[m2Temp1['sensorName'] == 'AuxTel-ESS01']                                        
outTemp1 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem0'],  start1, end1, index=301)
wind1 = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                ['speed'],  start1, end1, index=301)
m2Temp2 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem3', 'sensorName'],  start2, end2, index=201)
m2Temp2 = m2Temp2[m2Temp2['sensorName'] == 'AuxTel-ESS01'] 
m2Temp2 = m2Temp2.shift(-7, freq='D')
outTemp2 = await client.select_time_series('lsst.sal.ESS.temperature', \
                                                ['temperatureItem0'],  start2, end2, index=301)
outTemp2 = outTemp2.shift(-7, freq='D')
wind2 = await client.select_time_series('lsst.sal.ESS.airFlow', \
                                                ['speed'],  start2, end2, index=301)
wind2 = wind2.shift(-7, freq='D')

# Now plot it.

In [ ]:
%matplotlib inline
fig, axs = plt.subplots(2,1,figsize=(5,10))
plt.subplots_adjust(hspace=0.3)
plt.suptitle("Impact of AuxTel painting on in-dome temperatures")
axs[0].set_title("Temperatures")
m2Temp1['temperatureItem3'].plot(ax=axs[0], color='blue', label = "DomeT, no paint")
m2Temp2['temperatureItem3'].plot(ax=axs[0], color='red', label = "DomeT(shifted), paint")
outTemp1['temperatureItem0'].plot(ax=axs[0], color='cyan', label = "Outside, no paint")
outTemp2['temperatureItem0'].plot(ax=axs[0], color='magenta', label = "Outside(shifted), paint")
axs[0].legend(loc='upper left')
axs[0].set_ylim(10,50)
axs[0].set_ylabel("Temp (C)")
axs[1].set_title("Wind Speed")
wind1['speed'].plot(ax=axs[1], color='blue', label = "No paint")
wind2['speed'].plot(ax=axs[1], color='red', label = "Paint")
axs[1].set_ylim(0,20)
axs[1].set_ylabel("Wind speed(m/s)")
axs[1].legend(loc='upper left')
plt.savefig("/home/c/cslage/u/AuxTel/dome/Painting_Impact_25Mar24.png")